In [12]:
import subprocess
import time

In [22]:
base = """
  context_everywhere n > 0;
  context_everywhere x0 != null ** x0.length == n ** (\\forall* int i=0..n; Perm(x0[i], write));
  {annotations}
int main(int n, int[n] x0{args}){{
    loop_invariant 0 <= i && i<= n;
    loop_invariant (\\forall int j=0..i; x0[j] == 0{sumsinv});
  for(int i=0; i<n;i++){{
    x0[i] = 0{sums};
  }}
}}
"""

annotation = "context_everywhere {name} != null ** {name}.length == n ** (\\forall* int i=0..n; Perm({name}[i], 1\\2));"
arg = ", int[n] {name}"
sumpart = "+ {name}[i]"
sumpartinv = "+ {name}[j]"

In [23]:
def generate(n: int)->str:
    anns = '\n  '.join(map(lambda i: annotation.format(name=f"x{i}"), range(1,n+1)))
    args = ''.join(map(lambda i: arg.format(name=f"x{i}"), range(1,n+1)))
    sumsinv = ''.join(map(lambda i: sumpartinv.format(name=f"x{i}"), range(1,n+1)))
    sumparts = ''.join(map(lambda i: sumpart.format(name=f"x{i}"), range(1,n+1)))
    
    
    return base.format(annotations=anns, args=args, sumsinv=sumsinv,  sums=sumparts)

In [24]:
print(generate(1))


  context_everywhere n > 0;
  context_everywhere x0 != null ** x0.length == n ** (\forall* int i=0..n; Perm(x0[i], write));
  context_everywhere x1 != null ** x1.length == n ** (\forall* int i=0..n; Perm(x1[i], 1\2));
int main(int n, int[n] x0, int[n] x1){
    loop_invariant 0 <= i && i<= n;
    loop_invariant (\forall int j=0..i; x0[j] == 0+ x1[j]);
  for(int i=0; i<n;i++){
    x0[i] = 0+ x1[i];
  }
}



In [25]:
filename = "data/quant{i}.pvl"
for i in range(1,10):
    with open(filename.format(i=i), 'w') as f:
        f.write(generate(i))

In [31]:
command = ["/home/lars/data/vercors/bin/vct", "-q", "--silicon-quiet"]
start = time.time()
res = subprocess.run(command + ["data/base.pvl"])
end = time.time()
base = end-start
print("base", base, "success" if res.returncode == 0 else "fail")


for i in range(1,10):
    continue
    fn = "data/quant{i}.pvl".format(i=i)
    start = time.time()
    res = subprocess.run(command + [fn])
    end = time.time()
    print(fn, end - start, "success" if res.returncode == 0 else "fail")

base 7.234036922454834 success


In [27]:
res = """
base 6.245700120925903 succes
data/quant1.pvl 9.778679370880127 succes
data/quant2.pvl 11.161180019378662 succes
data/quant3.pvl 12.437653064727783 succes
data/quant4.pvl 14.520464658737183 succes
data/quant5.pvl 15.210931777954102 succes
data/quant6.pvl 18.143626928329468 succes
data/quant7.pvl 30.85934352874756 succes
data/quant8.pvl 57.702096462249756 succes
data/quant9.pvl 138.3834776878357 succes
"""
print(res)


base 6.245700120925903 succes
data/quant1.pvl 9.778679370880127 succes
data/quant2.pvl 11.161180019378662 succes
data/quant3.pvl 12.437653064727783 succes
data/quant4.pvl 14.520464658737183 succes
data/quant5.pvl 15.210931777954102 succes
data/quant6.pvl 18.143626928329468 succes
data/quant7.pvl 30.85934352874756 succes
data/quant8.pvl 57.702096462249756 succes
data/quant9.pvl 138.3834776878357 succes



In [67]:
base_p = """
resource hide_perm(int[] data, int i, int n) = data != null ** data.length == n ** (\\forall* int i; 0<=i && i< data.length; Perm(data[i], write));

  context_everywhere n > 0;
  context_everywhere hide_perm(x0, 0, n);
  {annotations}
int main(int n, int[n] x0{args}){{
    loop_invariant 0 <= i && i<= n;
    loop_invariant (\\forall int j=0..i; (\\unfolding hide_perm(x0, 0, n) \\in x0[j]) == 0{sumsinv});
  for(int i=0; i<n;i++){{
    {assigns}
    unfold hide_perm(x0, 0, n);
    x0[i] = 0{sums};
    fold hide_perm(x0, 0, n);
  }}
}}
"""

assign = "unfold hide_perm({name}, 0, n);\n    int {aname} = {name}[i];\n    fold hide_perm({name}, 0, n);\n    "
annotation_p = "context_everywhere hide_perm({name}, 0, n);"
arg_p = ", int[n] {name}"
sumpart_p = "+ {aname}"
sumpartinv_p = "+ (\\unfolding hide_perm({name}, 0, n) \\in {name}[j])"

In [68]:
def generate_p(n: int)->str:
    anns = '\n  '.join(map(lambda i: annotation_p.format(name=f"x{i}"), range(1,n+1)))
    args = ''.join(map(lambda i: arg_p.format(name=f"x{i}"), range(1,n+1)))
    sumsinv = ''.join(map(lambda i: sumpartinv_p.format(name=f"x{i}"), range(1,n+1)))
    sumparts = ''.join(map(lambda i: sumpart_p.format(name=f"x{i}", aname=f"a{i}"), range(1,n+1)))
    assigns = ''.join(map(lambda i: assign.format(name=f"x{i}", aname=f"a{i}"), range(1,n+1)))
    
    
    return base_p.format(annotations=anns, args=args, sumsinv=sumsinv,  sums=sumparts, assigns=assigns)

In [69]:
print(generate_p(2))


resource hide_perm(int[] data, int i, int n) = data != null ** data.length == n ** (\forall* int i; 0<=i && i< data.length; Perm(data[i], write));

  context_everywhere n > 0;
  context_everywhere hide_perm(x0, 0, n);
  context_everywhere hide_perm(x1, 0, n);
  context_everywhere hide_perm(x2, 0, n);
int main(int n, int[n] x0, int[n] x1, int[n] x2){
    loop_invariant 0 <= i && i<= n;
    loop_invariant (\forall int j=0..i; (\unfolding hide_perm(x0, 0, n) \in x0[j]) == 0+ (\unfolding hide_perm(x1, 0, n) \in x1[j])+ (\unfolding hide_perm(x2, 0, n) \in x2[j]));
  for(int i=0; i<n;i++){
    unfold hide_perm(x1, 0, n);
    int a1 = x1[i];
    fold hide_perm(x1, 0, n);
    unfold hide_perm(x2, 0, n);
    int a2 = x2[i];
    fold hide_perm(x2, 0, n);
    
    unfold hide_perm(x0, 0, n);
    x0[i] = 0+ a1+ a2;
    fold hide_perm(x0, 0, n);
  }
}



In [73]:
filename_p = "data/quant_p{i}.pvl"
for i in range(1,20):
    with open(filename_p.format(i=i), "w") as f:
        f.write(generate_p(i))

In [72]:
command = ["/home/lars/data/vercors/bin/vct", "-q", "--silicon-quiet"]

for i in range(1,10):
    fn = filename_p.format(i=i)
    start = time.time()
    res = subprocess.run(command + [fn])
    end = time.time()
    print(fn, end - start, "success" if res.returncode == 0 else "fail")

data/quant_p1.pvl 6.917606592178345 success
data/quant_p2.pvl 7.450190305709839 success
data/quant_p3.pvl 8.49233102798462 success
data/quant_p4.pvl 9.079141855239868 success
data/quant_p5.pvl 10.259852409362793 success
data/quant_p6.pvl 11.947216033935547 success
data/quant_p7.pvl 13.722721099853516 success
data/quant_p8.pvl 15.532447814941406 success
data/quant_p9.pvl 19.215627431869507 success


In [ ]:
command = ["/home/lars/data/vercors/bin/vct", "-q", "--silicon-quiet"]

for i in range(16,18):
    fn = "data/quant_p{i}.pvl".format(i=i)
    start = time.time()
    res = subprocess.run(command + [fn])
    end = time.time()
    print(fn, end - start, "success" if res.returncode == 0 else "fail")

In [ ]:
res_p = """
data/quant_p1.pvl 6.917606592178345 success
data/quant_p2.pvl 7.450190305709839 success
data/quant_p3.pvl 8.49233102798462 success
data/quant_p4.pvl 9.079141855239868 success
data/quant_p5.pvl 10.259852409362793 success
data/quant_p6.pvl 11.947216033935547 success
data/quant_p7.pvl 13.722721099853516 success
data/quant_p8.pvl 15.532447814941406 success
data/quant_p9.pvl 19.215627431869507 success
data/quant_p10.pvl 23.15853476524353 success
data/quant_p11.pvl 24.15439200401306 success
data/quant_p12.pvl 30.964784145355225 success
data/quant_p13.pvl 50.016061782836914 success
data/quant_p14.pvl 57.36567568778992 success
data/quant_p15.pvl 71.61014318466187 success
"""

In [78]:
base_p_u = """
resource hide_perm(int[] data, int i, int n) = data != null ** data.length == n ** (\\forall* int i; 0<=i && i< data.length; Perm(data[i], write));

  context_everywhere n > 0;
  context_everywhere hide_perm(x0, 0, n);
  {annotations}
int main(int n, int[n] x0{args}){{
    loop_invariant 0 <= i && i<= n;
    loop_invariant (\\forall int j=0..i; (\\unfolding hide_perm(x0, 0, n) \\in x0[j]) == 0{sumsinv});
  for(int i=0; i<n;i++){{
    {assigns}
    unfold hide_perm(x0, 0, n);
    x0[i] = 0{sums};
    fold hide_perm(x0, 0, n);
  }}
}}
"""

assign_p_u = "unfold hide_perm({name}, {i}, n);\n    int {aname} = {name}[i];\n    fold hide_perm({name}, {i}, n);\n    "
annotation_p_u = "context_everywhere hide_perm({name}, {i}, n);"
arg_p_u = ", int[n] {name}"
sumpart_p_u = "+ {aname}"
sumpartinv_p_u = "+ (\\unfolding hide_perm({name}, {i}, n) \\in {name}[j])"

In [83]:
def generate_p_u(n: int)->str:
    anns = '\n  '.join(map(lambda i: annotation_p_u.format(name=f"x{i}", i=i), range(1,n+1)))
    args = ''.join(map(lambda i: arg_p_u.format(name=f"x{i}"), range(1,n+1)))
    sumsinv = ''.join(map(lambda i: sumpartinv_p_u.format(name=f"x{i}", i=i), range(1,n+1)))
    sumparts = ''.join(map(lambda i: sumpart_p_u.format(name=f"x{i}", aname=f"a{i}", i=i), range(1,n+1)))
    assigns = ''.join(map(lambda i: assign_p_u.format(name=f"x{i}", aname=f"a{i}", i=i), range(1,n+1)))
    
    
    return base_p_u.format(annotations=anns, args=args, sumsinv=sumsinv,  sums=sumparts, assigns=assigns)

In [84]:
filename_p = "data/quant_p_u{i}.pvl"
for i in range(1,20):
    with open(filename_p.format(i=i), "w") as f:
        f.write(generate_p_u(i))

In [ ]:
command = ["/home/lars/data/vercors/bin/vct", "-q", "--silicon-quiet"]

for i in range(13,18):
    fn = filename_p.format(i=i)
    start = time.time()
    res = subprocess.run(command + [fn])
    end = time.time()
    print(fn, end - start, "success" if res.returncode == 0 else "fail")

In [ ]:
res_p_u =
"""
data/quant_p_u1.pvl 6.92410945892334 success
data/quant_p_u2.pvl 7.535781621932983 success
data/quant_p_u3.pvl 8.280990839004517 success
data/quant_p_u4.pvl 9.366011142730713 success
data/quant_p_u5.pvl 10.714183568954468 success
data/quant_p_u6.pvl 12.295013189315796 success
data/quant_p_u7.pvl 13.86497187614441 success
data/quant_p_u8.pvl 15.526273727416992 success
data/quant_p_u9.pvl 18.14137887954712 success
data/quant_p_u10.pvl 21.360657930374146 success
data/quant_p_u11.pvl 24.72795343399048 success
data/quant_p_u12.pvl 36.47441625595093 success
"""

In [90]:
base_p_u2 = """
resource hide_perm0(int[] data, int n) = data != null ** data.length == n ** (\\forall* int i; 0<=i && i< data.length; Perm(data[i], write));
{preds}

  context_everywhere n > 0;
  context_everywhere hide_perm0(x0, n);
  {annotations}
int main(int n, int[n] x0{args}){{
    loop_invariant 0 <= i && i<= n;
    loop_invariant (\\forall int j=0..i; (\\unfolding hide_perm0(x0, n) \\in x0[j]) == 0{sumsinv});
  for(int i=0; i<n;i++){{
    {assigns}
    unfold hide_perm0(x0, n);
    x0[i] = 0{sums};
    fold hide_perm0(x0, n);
  }}
}}
"""

pred_p_u2 = "resource hide_perm{i}(int[] data, int n) = data != null ** data.length == n ** (\\forall* int i; 0<=i && i< data.length; Perm(data[i], write));\n"
assign_p_u2 = "unfold hide_perm{i}({name}, n);\n    int {aname} = {name}[i];\n    fold hide_perm{i}({name}, n);\n    "
annotation_p_u2 = "context_everywhere hide_perm{i}({name}, n);"
arg_p_u2 = ", int[n] {name}"
sumpart_p_u2 = "+ {aname}"
sumpartinv_p_u2 = "+ (\\unfolding hide_perm{i}({name}, n) \\in {name}[j])"

In [91]:
def generate_p_u2(n: int)->str:
    anns = '\n  '.join(map(lambda i: annotation_p_u2.format(name=f"x{i}", i=i), range(1,n+1)))
    args = ''.join(map(lambda i: arg_p_u2.format(name=f"x{i}"), range(1,n+1)))
    sumsinv = ''.join(map(lambda i: sumpartinv_p_u2.format(name=f"x{i}", i=i), range(1,n+1)))
    sumparts = ''.join(map(lambda i: sumpart_p_u2.format(name=f"x{i}", aname=f"a{i}", i=i), range(1,n+1)))
    assigns = ''.join(map(lambda i: assign_p_u2.format(name=f"x{i}", aname=f"a{i}", i=i), range(1,n+1)))
    preds = ''.join(map(lambda i: pred_p_u2.format(i=i), range(1,n+1)))
    
    
    return base_p_u2.format(annotations=anns, args=args, sumsinv=sumsinv,  sums=sumparts, assigns=assigns, preds=preds)

In [92]:
filename_p = "data/quant_p_u_v2_{i}.pvl"
for i in range(1,20):
    with open(filename_p.format(i=i), "w") as f:
        f.write(generate_p_u2(i))

In [93]:
command = ["/home/lars/data/vercors/bin/vct", "-q", "--silicon-quiet"]

for i in range(1,20):
    fn = filename_p.format(i=i)
    start = time.time()
    res = subprocess.run(command + [fn])
    end = time.time()
    print(fn, end - start, "success" if res.returncode == 0 else "fail")

data/quant_p_u_v2_1.pvl 6.958630561828613 success
data/quant_p_u_v2_2.pvl 7.383204221725464 success
data/quant_p_u_v2_3.pvl 7.864865303039551 success
data/quant_p_u_v2_4.pvl 8.347679376602173 success
data/quant_p_u_v2_5.pvl 8.85740613937378 success
data/quant_p_u_v2_6.pvl 9.241874933242798 success
data/quant_p_u_v2_7.pvl 10.591420412063599 success
data/quant_p_u_v2_8.pvl 11.159575939178467 success
data/quant_p_u_v2_9.pvl 12.496795177459717 success
data/quant_p_u_v2_10.pvl 15.170531034469604 success
data/quant_p_u_v2_11.pvl 14.941126108169556 success
data/quant_p_u_v2_12.pvl 37.044289350509644 success
data/quant_p_u_v2_13.pvl 32.95157074928284 success
data/quant_p_u_v2_14.pvl 34.14073944091797 success
data/quant_p_u_v2_15.pvl 71.15804743766785 success
data/quant_p_u_v2_16.pvl 73.67194962501526 success
At data/quant_p_u_v2_17.pvl
--------------------------------------
   40    context_everywhere hide_perm17(x17, n);
   41  int main(int n, int[n] x0, int[n] x1, int[n] x2, int[n] x3, int[n

Exception in thread "[VerCors] Shutdown hook to abort progress on exit" java.lang.IllegalStateException: Timer already cancelled.
	at java.base/java.util.Timer.sched(Timer.java:409)
	at java.base/java.util.Timer.schedule(Timer.java:205)
	at hre.progress.Progress$.hre$progress$Progress$$delayNextUpdate(Progress.scala:57)
	at hre.progress.Progress$.update(Progress.scala:66)
	at hre.progress.task.AbstractTask.end(AbstractTask.scala:92)
	at hre.progress.task.AbstractTask.abort(AbstractTask.scala:101)
	at hre.progress.task.AbstractTask.$anonfun$abort$1(AbstractTask.scala:100)
	at hre.progress.task.AbstractTask.$anonfun$abort$1$adapted(AbstractTask.scala:100)
	at scala.collection.immutable.Vector.foreach(Vector.scala:2124)
	at hre.progress.task.AbstractTask.abort(AbstractTask.scala:100)
	at hre.progress.task.AbstractTask.$anonfun$abort$1(AbstractTask.scala:100)
	at hre.progress.task.AbstractTask.$anonfun$abort$1$adapted(AbstractTask.scala:100)
	at scala.collection.immutable.Vector.foreach(Ve

KeyboardInterrupt: 

# Viper

In [28]:
base_vpr = """
domain Array  {{
  
  function array_loc(a: Array, i: Int): Ref 
  
  function alen(a: Array): Int 
  
  function loc_inv_1(loc: Ref): Array 
  
  function loc_inv_2(loc: Ref): Int 
  
  axiom {{
    (forall a: Array, i: Int ::
      {{ array_loc(a, i) }}
      loc_inv_1(array_loc(a, i)) == a && loc_inv_2(array_loc(a, i)) == i)
  }}
  
  axiom {{
    (forall a: Array :: {{ alen(a) }} alen(a) >= 0)
  }}
}}

field int: Int

function aloc(a: Array, i: Int): Ref
  requires 0 <= i
  requires i < alen(a)
  decreases 
  ensures loc_inv_1(result) == a
  ensures loc_inv_2(result) == i
{{
  array_loc(a, i)
}}

method main1(tid: Int, n: Int, x0: Array{args})
  requires 0 < n
  requires alen(x0) == n
  requires (forall i: Int ::
      {{ aloc(x0, i) }}
      0 <= i && i < n ==> acc(aloc(x0, i).int, write))
  {reqs}
  ensures 0 < n
  ensures alen(x0) == n
  ensures (forall i: Int ::
      {{ aloc(x0, i) }}
      0 <= i && i < n ==> acc(aloc(x0, i).int, write))
  {enss}
{{
  {{
    var exc: Ref
    var i: Int
    var excbeforeloop: Ref
    exc := null
    excbeforeloop := exc
    i := 0
    while (i < n)
      invariant exc == excbeforeloop
      invariant 0 < n
  
      invariant alen(x0) == n
      invariant (forall i1: Int ::
          {{ aloc(x0, i1) }}
          0 <= i1 && i1 < n ==> acc(aloc(x0, i1).int, write))
  
      {invs}
      invariant 0 <= i
      invariant i < n + 1
      invariant (forall j: Int ::
          {{ aloc(x0, j) }}
          0 <= j && j < i ==>
          aloc(x0, j).int == 0{inv_sums})
    {{
      aloc(x0, i).int :=  0{loop_sums}
      i := i + 1
    }}
    assert exc == null
  }}
}}
"""

arg = """, {x}: Array"""
req = """requires alen({x}) == n
  requires (forall i: Int ::
      {{ aloc({x}, i) }}
      0 <= i && i < n ==> acc(aloc({x}, i).int, 1 * write / 2))"""
ens = """ensures alen({x}) == n
  ensures (forall i: Int ::
      {{ aloc({x}, i) }}
      0 <= i && i < n ==> acc(aloc({x}, i).int, 1 * write / 2))"""
inv = """invariant alen({x}) == n
      invariant (forall i1: Int ::
          {{ aloc({x}, i1) }}
          0 <= i1 && i1 < n ==>
          acc(aloc({x}, i1).int, 1 * write / 2))
"""
inv_sum = """ + aloc({x}, j).int"""
loop_sum = """ + aloc({x}, i).int"""

In [29]:
def generate_viper(n: int)->str:
    args = ''.join(map(lambda i: arg.format(x=f"x{i}"), range(1,n+1)))
    reqs = '\n  '.join(map(lambda i: req.format(x=f"x{i}"), range(1,n+1)))
    enss = '\n  '.join(map(lambda i: ens.format(x=f"x{i}"), range(1,n+1)))
    invs = '\n    '.join(map(lambda i: inv.format(x=f"x{i}"), range(1,n+1)))
    inv_sums = ''.join(map(lambda i: inv_sum.format(x=f"x{i}"), range(1,n+1)))
    loop_sums = ''.join(map(lambda i: loop_sum.format(x=f"x{i}"), range(1,n+1)))
    
    return base_vpr.format(args=args, reqs=reqs, enss=enss, invs=invs, inv_sums=inv_sums,  loop_sums=loop_sums)

In [30]:
print(generate_viper(10))


domain Array  {
  
  function array_loc(a: Array, i: Int): Ref 
  
  function alen(a: Array): Int 
  
  function loc_inv_1(loc: Ref): Array 
  
  function loc_inv_2(loc: Ref): Int 
  
  axiom {
    (forall a: Array, i: Int ::
      { array_loc(a, i) }
      loc_inv_1(array_loc(a, i)) == a && loc_inv_2(array_loc(a, i)) == i)
  }
  
  axiom {
    (forall a: Array :: { alen(a) } alen(a) >= 0)
  }
}

field int: Int

function aloc(a: Array, i: Int): Ref
  requires 0 <= i
  requires i < alen(a)
  decreases 
  ensures loc_inv_1(result) == a
  ensures loc_inv_2(result) == i
{
  array_loc(a, i)
}

method main1(tid: Int, n: Int, x0: Array, x1: Array, x2: Array, x3: Array, x4: Array, x5: Array, x6: Array, x7: Array, x8: Array, x9: Array, x10: Array)
  requires 0 < n
  requires alen(x0) == n
  requires (forall i: Int ::
      { aloc(x0, i) }
      0 <= i && i < n ==> acc(aloc(x0, i).int, write))
  requires alen(x1) == n
  requires (forall i: Int ::
      { aloc(x1, i) }
      0 <= i && i < n ==> 

In [32]:
filename = "data/quant{i}.vpr"
for i in range(1,20):
    with open(filename.format(i=i), 'w') as f:
        f.write(generate_viper(i))

In [16]:
silicon_path = "/home/lars/data/silicon/target/scala-2.13/silicon.jar"
command = ["java",
            "-Xss1024m",
            "-Xmx4024m",
            "-jar",
            silicon_path,
            "--logLevel",
            "OFF"
            ]

env = {"Z3_EXE" : "/usr/bin/z3"}

for i in range(1,10):
    fn = "data/quant{i}.vpr".format(i=i)
    start = time.time()
    res = subprocess.run(command + [fn], env=env)
    end = time.time()
    print(fn, end - start, "success" if res.returncode == 0 else "fail")

Silicon 1.1-SNAPSHOT (5e3ab014+)
Silicon finished verification successfully in 4.18s.
data/quant1.vpr 4.536563873291016 success
Silicon 1.1-SNAPSHOT (5e3ab014+)
Silicon finished verification successfully in 4.77s.
data/quant2.vpr 5.160653829574585 success
Silicon 1.1-SNAPSHOT (5e3ab014+)
Silicon finished verification successfully in 5.19s.
data/quant3.vpr 5.564464092254639 success
Silicon 1.1-SNAPSHOT (5e3ab014+)
Silicon finished verification successfully in 5.82s.
data/quant4.vpr 6.1763715744018555 success
Silicon 1.1-SNAPSHOT (5e3ab014+)
Silicon finished verification successfully in 6.96s.
data/quant5.vpr 7.328972816467285 success
Silicon 1.1-SNAPSHOT (5e3ab014+)
Silicon finished verification successfully in 10.68s.
data/quant6.vpr 11.023087501525879 success
Silicon 1.1-SNAPSHOT (5e3ab014+)
Silicon finished verification successfully in 24.24s.
data/quant7.vpr 24.652435064315796 success
Silicon 1.1-SNAPSHOT (5e3ab014+)
Silicon finished verification successfully in 01m:30s.
data/quant8

In [ ]:
silicon_path = "/home/lars/data/silicon/target/scala-2.13/silicon.jar"
command = ["java",
            "-Xss1024m",
            "-Xmx4024m",
            "-jar",
            silicon_path,
            "--logLevel",
            "OFF"
            ]

env = {"Z3_EXE" : "/usr/bin/z3"}

for i in range(10,20):
    fn = "data/quant{i}.vpr".format(i=i)
    start = time.time()
    res = subprocess.run(command + [fn], env=env)
    end = time.time()
    print(fn, end - start, "success" if res.returncode == 0 else "fail")

Silicon 1.1-SNAPSHOT (5e3ab014+)


In [ ]:
carbon_path = "/home/lars/data/carbon/target/scala-2.13/carbon.jar"
command = ["java",
            "-Xss1024m",
            "-Xmx4024m",
            "-jar",
            carbon_path
            ]

env = {"Z3_EXE" : "/usr/bin/z3", "BOOGIE_EXE" : "/home/lars/.config/Code/User/globalStorage/viper-admin.viper/Stable/ViperTools/boogie/Binaries/Boogie"}

for i in range(1,10):
    fn = "data/quant{i}.vpr".format(i=i)
    start = time.time()
    res = subprocess.run(command + [fn], env=env)
    end = time.time()
    print(fn, end - start, "success" if res.returncode == 0 else "fail")

## End of notebook